In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import requests
import json
import time
# import the library
import googletrans
from googletrans import Translator
import goslate
from datetime import datetime
Token = 'yoVFkAKoWYoOcIqPtXRBolQVwygTyqin'

In [2]:
latest_data=pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province.csv')

In [3]:
weather=pd.read_csv('C:\ML\COV files\citiesIT.csv')

In [6]:
lockdown=pd.read_csv('https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=2dcf78defb92930a')
lockdown=lockdown[['country_region','sub_region_1','date','retail_and_recreation_percent_change_from_baseline','grocery_and_pharmacy_percent_change_from_baseline',
                'transit_stations_percent_change_from_baseline' ,'workplaces_percent_change_from_baseline' ,'residential_percent_change_from_baseline']]

lockdown.columns = ['country','state',
                    'date','retail_and_recreation','grocery_and_pharnacy'
                    ,'transit_stations' ,'workplaces' ,'residential']



c:\users\sastry\appdata\local\programs\python\python35\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def translateNames(row):
    gs = goslate.Goslate()
    time.sleep(1)
    translatedText = gs.translate(row,'it')
    return translatedText

In [22]:
latest_data=latest_data[['data','denominazione_regione','denominazione_provincia','sigla_provincia','totale_casi']]

In [23]:
latest_data.columns=['date','state','city','state_abbv','total_cases']

In [30]:
def changeDateFormat(row):
    datetime_object = datetime.strptime(row, '%Y-%m-%d')
    return (datetime_object)

In [37]:
# latest_data['date']=latest_data['date'].apply(changeDateFormat)
latest_data['date']=pd.to_datetime(latest_data['date'])

c:\users\sastry\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [38]:
latest_data['date'] = latest_data['date'].dt.normalize()

c:\users\sastry\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:

lockdown.replace(to_replace ="Apulia", value ="Puglia", inplace=True)
lockdown.replace(to_replace ="Lombary", value ="Lombardia", inplace=True)
lockdown.replace(to_replace ="Piedmont", value ="Piemonte", inplace=True)
lockdown.replace(to_replace ="Sardinia", value ="Sardegna", inplace=True)
lockdown.replace(to_replace ="Sicily", value ="Sicilia", inplace=True)
lockdown.replace(to_replace ="Trentino-South Tyrol", value ="Trentino Alto Adige", inplace=True)
lockdown.replace(to_replace ="Tuscany-", value ="Toscana", inplace=True)


In [12]:
df1=pd.merge(weather,lockdown)

In [13]:
temp_df=pd.DataFrame()
df2=pd.DataFrame()

In [14]:
def getWeatherData(weatherID,city):
    # initialize lists to store data
    dates_temp = []
    dates_prcp = []
    temps = []
    prcp = []
    year = str(2020)
    # make the api call
    r = requests.get(
        'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&units=metric&limit=1000&stationid=GHCND:'+weatherID+'&startdate=' + year + '-01-01&enddate=' + year + '-12-31',
        headers={'token': Token})
    print (r)
    # load the api response as a json
    d = json.loads(r.text)
    # get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype'] == 'TAVG']
    # get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    # get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]
    #initialize dataframe
    df_temp = pd.DataFrame()
    #populate date and average temperature fields (cast string date to datetime and convert temperature from tenths of Celsius to Fahrenheit)
    df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
    df_temp['avgTemp'] = [float(v) for v in temps]
    df_temp['date']=df_temp['date'].astype(str)
    df_temp['city']=str(city)
    
    return df_temp

In [17]:
for i in range(0, len(weather)):
#     print (weather.iat[i, 3])
    temp_df=getWeatherData(weather.iat[i, 3],weather.iat[i,0])
    last=pd.merge(temp_df,df1)
    df2=pd.concat([df2,last])
    last.empty

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [32]:
df2['date']=df2['date'].apply(changeDateFormat)

In [42]:
df3=pd.merge(latest_data,df2)

In [66]:
for row in range(1, len(df3)):
    new=abs(df3.iat[row,4]-df3.iat[row-1,4])
    df3['new_cases'].loc[row]=new
    

c:\users\sastry\appdata\local\programs\python\python35\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [72]:
df3.to_csv('test.csv')